In [ ]:
import os

import torch
from PIL import Image
from pipelines.inverted_ve_pipeline import create_image_grid
from pipelines.pipeline_stable_diffusion_xl import StableDiffusionXLPipeline
from utils import init_latent, memory_efficient


device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    torch_dtype = torch.float32
else:
    torch_dtype = torch.float16

In [ ]:

def create_number_list(n):
    return list(range(n + 1))


def create_nested_list(t):
    return [[0, t]]


def create_prompt(style_name):
    pre_prompt_dicts = {
        "kids drawing": (
            "kids drawing of {prompt}. crayon, colored pencil, marker",
            "",
        ),
        "self portrait": ("{prompt} of van gogh", ""),
        "Sunflowers": ("{prompt} of van gogh", ""),
        "The kiss": ("{prompt} of gustav klimt", ""),
        "Vitruvian Man": ("{prompt} of leonardo da vinci", ""),
        "Weeping woman": ("{prompt} of pablo picasso", ""),
        "The scream": ("{prompt} of edvard munch", ""),
        "The starry night": ("{prompt} of van gogh", ""),
        "Starry night over the rhone": ("{prompt} of van gogh", ""),
        "MOVIE": ("{prompt} movie poster", ""),
    }

    if style_name in pre_prompt_dicts.keys():
        return pre_prompt_dicts[style_name]
    else:
        return ("{prompt}", "")  # base_prompt, negative_prompt


In [ ]:
from dataclasses import dataclass


@dataclass
class Arguments:
    tar_obj: str = "a dog and a man" # target object
    guidance_scale: float = 7.0
    output_num: int = 4
    activate_step: int = 50
    results_dir: str = "./results"

In [ ]:
from diffusers import DDIMScheduler


args = Arguments()
tar_seeds = create_number_list(args.output_num)
activate_step_indices = create_nested_list(args.activate_step)


pipe = StableDiffusionXLPipeline.from_pretrained(
    "RunDiffusion/Juggernaut-X-v10",
    torch_dtype=torch_dtype,
    variant="fp16",
)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

str_activate_layer, str_activate_step = pipe.activate_layer(
    activate_layer_indices=[[0, 0], [128, 140]],
    attn_map_save_steps=[],
    activate_step_indices=activate_step_indices,
    use_shared_attention=False,
)

In [ ]:
from diffusers.utils import load_image


img_path = args.img_path
tar_obj = args.tar_obj
guidance_scale = args.guidance_scale

with torch.no_grad():
    hf_dataset_base_url = "https://huggingface.co/datasets/GoGiants1/TMDBEval500/resolve/main/TMDBEval500/images/"
    real_img = load_image(hf_dataset_base_url + '1.jpg')

    style_name = "MOVIE"
    latents = []

    base_prompt, negative_prompt = create_prompt(style_name)
    ref_prompt = base_prompt.replace("{prompt}", style_name)
    inf_prompt = base_prompt.replace("{prompt}", tar_obj)

    for tar_seed in tar_seeds:
        latents.append(
            init_latent(
                model=pipe, device_name=device, dtype=torch_dtype, seed=tar_seed
            )
        )

    latents = torch.cat(latents, dim=0)

    images = pipe(
        prompt=ref_prompt,
        guidance_scale=7,
        latents=latents,
        num_images_per_prompt=len(tar_seeds),
        target_prompt=inf_prompt,
        use_inf_negative_prompt=False,
        use_advanced_sampling=False,
        use_prompt_as_null=True,
        image=real_img,
    )[0]
    # [real image, fake1, fake2, ... ]
    save_path = os.path.join(args.results_dir, "{}_{}.png".format(style_name, tar_obj))

    n_row = 1
    n_col = len(tar_seeds)
    grid = create_image_grid(images, n_row, n_col)

    grid.save(save_path)
    print(f"saved to {save_path}")
